# Using the SEBA Algorithm together with Linear Response for the Ocean System (SEBA Part 2)

This is the second part of a two notebook discussion of using SEBA together with Linear Response in Julia. In the first part, we illustrated the ideas on the conceptionally simpler Double Gyre System, and now we will try to apply this to a system determined by real world ocean data. As parameter we will use the **stoptime**, just as in the Double Gyre example, not some more sophisticated parameter like wind etc., in order to increase the difficulty just one step at a time.

## Setup

In [ ]:
include("../../src/CoherentStructures.jl")

In [ ]:
using StreamMacros, Main.CoherentStructures
using Tensors, Arpack, JLD2, OrdinaryDiffEq
using Plots

In [ ]:
JLD2.@load("Ocean_geostrophic_velocity.jld2");
UV = interpolateVF(Lon, Lat, Time, UT, VT);

In [ ]:
# Parameters
grid_resolution        = 100
quadrature_order       = 5
ϵ                      = 10               # perturbation
nev                    = 10               # number of eigenvalues
t_initial              = minimum(Time)  
t_end                  = t_initial+70     # parameter value corresponding to ϵ=0
solver_tolerance       = 1e-8
solver                 = OrdinaryDiffEq.BS5();

In [ ]:
T(x, param) = flow(interp_rhs, x, [t_initial, t_end + param], p=UV,
        tolerance=solver_tolerance,solver=solver)[end]

In [ ]:
# standard domain from the literature
LL, UR = (-4.0, -34.0), (6.0, -28.0)
lon_resolution = grid_resolution
lat_resolution = Integer(floor((UR[2] - LL[2])/(UR[1] - LL[1])*grid_resolution))
ctx, _ = regularTriangularGrid((lon_resolution, lat_resolution), 
        LL, UR, quadrature_order = quadrature_order);
# homogeneous Dirichlet boundary conditions to avoid intersection with the boundary of the domain
bdata = getHomDBCS(ctx, "all");

## Compute Eigenfunctions and Linear Response

In [ ]:
M = assembleMassMatrix(ctx);

In [ ]:
# assemble K
DT₀(x) = linearized_flow_autodiff(y -> T(y,0) , x)
A₀(x) = 0.5*(one(Tensor{2,2}) + dott(inv(DT₀(x))))
@time K = assembleStiffnessMatrix(ctx, A₀, bdata=bdata);

In [ ]:
DTϵ(x) = linearized_flow_autodiff(y -> T(y,ϵ) , x)
Aϵ(x) = 0.5*(one(Tensor{2,2}) + dott(inv(DTϵ(x))))
Kϵ = assembleStiffnessMatrix(ctx, Aϵ, bdata=bdata);

In [ ]:
# assemble the linear response matrix L
Adot = x -> linear_response_tensor(T, x, 0)
@time L = assembleStiffnessMatrix(ctx, Adot, bdata=bdata);

In [ ]:
normalizeUL2(u,M) = u.*sign.(sum(u))./ sqrt(u'*M*u);
λ₀, u₀ = eigs(K, M, which=:SM, nev=nev)
@assert all(imag.(u₀) .== 0)
@assert all(imag.(λ₀) .== 0)
u₀ = real.(u₀)
λ₀ = real.(λ₀)
for i in 1:size(u₀)[2]
    u₀[:,i] = normalizeUL2(u₀[:,i],M)
end

λϵ, uϵ = eigs(Kϵ, M, which=:SM, nev=nev)
@assert all(imag.(uϵ) .== 0)
@assert all(imag.(λϵ) .== 0)
uϵ = real.(uϵ)
λϵ = real.(λϵ)
for i in 1:size(uϵ)[2]
    uϵ[:,i] = normalizeUL2(uϵ[:,i],M)
end

u_dot = zero(u₀)
λ_dot = zero(λ₀)
for i in 1:size(u₀)[2] 
    u_dot[:,i], λ_dot[i] = getLinearResponse(u₀[:,i],λ₀[i],M,K,L)
end

## Some first plots

In [ ]:
#select number of evs
Plots.scatter(1:nev,λ₀)

In [ ]:
#Eigengap after 5 eigenvalues
nev = 5
u₀ = u₀[:,1:nev]
λ₀ = λ₀[1:nev]
uϵ = uϵ[:,1:nev]
λϵ = λϵ[1:nev]
u_dot = u_dot[:,1:nev]
λ_dot = λ_dot[1:nev];

In [ ]:
for i in 2:nev
    print("exact ev"*string(i)*": "*string(λϵ[i])*
        "\t predicted ev"*string(i)*": "*string(λ₀[i] + ϵ*λ_dot[i])*
        "\t relative error: "*string(abs(λ₀[i] + ϵ*λ_dot[i] - λϵ[i])/abs(λϵ[i]))*"\n")
end

In [ ]:
color =    :balance
xticks =   -4:2:6
yticks =   -34:2:-28
colorbar = :left
cmins =     [-1.0, -1.0, -1.0, -1.0,-1.0]
cmaxs =     -cmins
cmins_lr =  [-0.2, -0.2, -0.2, -0.2, -0.2]
cmaxs_lr =  -cmins_lr;

In [ ]:
plots = []
for i in 1:nev
    push!(plots,plot_u(ctx, u₀[:,i],  lon_resolution, lat_resolution, bdata=bdata,
            title="u₀"*string(i), 
                colorbar=colorbar, color=color, clims=(cmins[i],cmaxs[i]), xticks=xticks, yticks=yticks))
    push!(plots,plot_u(ctx, u_dot[:,i],  lon_resolution, lat_resolution, bdata=bdata,
            title="̇u₀"*string(i), 
        colorbar=colorbar, color=color, clims=(cmins_lr[i],cmaxs_lr[i]), xticks=xticks, yticks=yticks))
    push!(plots,plot_u(ctx, u₀[:,i] + ϵ*u_dot[:,i],  lat_resolution, grid_resolution, bdata=bdata,
            title="u₀"*string(i)*" + $(ϵ)u̇ ₀"*string(i), 
    colorbar=colorbar, color=color, clims=(cmins[i],cmaxs[i]), xticks=xticks, yticks=yticks))
    push!(plots,plot_u(ctx, uϵ[:,i],  lon_resolution, lat_resolution, bdata=bdata,
            title="uϵ"*string(i), 
            colorbar=colorbar, color=color, clims=(cmins[i],cmaxs[i]), xticks=xticks, yticks=yticks))
end
Plots.plot(plots..., fmt=:png, dpi=500,layout=grid(nev,4),
            xtickfontsize=2,ytickfontsize=2,xguidefontsize=2,yguidefontsize=2,legendfontsize=5,titlefontsize=5)

In [ ]:
#limit ourselves to one ev for plotting etc.
ev = 2

In [ ]:
plots = []
push!(plots,plot_u(ctx, u₀[:,ev],  grid_resolution, grid_resolution, 
        title="u₀"*string(ev), 
            colorbar=colorbar, color=color, clims=(cmins[ev],cmaxs[ev]), xticks=xticks, yticks=yticks))
push!(plots,plot_u(ctx, u_dot[:,ev],  grid_resolution, grid_resolution, 
        title="̇u₀"*string(ev), 
    colorbar=colorbar, color=color, clims=(cmins_lr[ev],cmaxs_lr[ev]), xticks=xticks, yticks=yticks))
push!(plots,plot_u(ctx, u₀[:,ev] + ϵ*u_dot[:,ev],  grid_resolution, grid_resolution, 
        title="u₀"*string(ev)*" + $(ϵ)u̇ ₀"*string(ev), 
colorbar=colorbar, color=color, clims=(cmins[ev],cmaxs[ev]), xticks=xticks, yticks=yticks))
push!(plots,plot_u(ctx, uϵ[:,ev],  grid_resolution, grid_resolution, 
        title="uϵ"*string(ev), 
        colorbar=colorbar, color=color, clims=(cmins[ev],cmaxs[ev]), xticks=xticks, yticks=yticks))
Plots.plot(plots..., fmt=:png, dpi=500,layout=grid(2,2),
            xtickfontsize=2,ytickfontsize=2,xguidefontsize=2,yguidefontsize=2,legendfontsize=5,titlefontsize=5)

## Contour Plots

In [ ]:
using Contour
using PyCall
using PyPlot
import Main.CoherentStructures:dynamic_cheeger_value
dynamic_cheeger_value(c) = dynamic_cheeger_value(ctx,c,x->T(x,0);tolerance=1e-10)
function plot_vectorfield(xs,ys,dx,dy,ax;scale=1,step=1)
    xs, ys = xs[1:step:end], ys[1:step:end]
    dx, dy = dx[1:step:end,1:step:end], dy[1:step:end,1:step:end]
    ax.quiver(xs,ys,dx',dy',scale=scale,color="blue")
end

In [ ]:
best_levelset, best_value = get_minimal_levelset(
    ctx,u₀[:,ev],dynamic_cheeger_value,
    n_candidates=1000,min=0)

c_best = level(best_levelset)

plt.figure(figsize=(7,7), dpi=100)
ax = plt.axes()
ax.set(xlim=(0, 1), ylim=(0, 1), xticks=(0,1), yticks=(0,1))
plot_vectorfield(get_levelset_evolution(ctx,u₀[:,ev],u_dot[:,ev])...,ax,scale=20,step=4)
curves           = lines(get_levelset(ctx, u₀[:,ev],c_best))
curves_epsilon   = lines(get_levelset(ctx, uϵ[:,ev],c_best))
curves_predicted = lines(get_levelset(ctx, u₀[:,ev] + ϵ*u_dot[:,ev],c_best))

for c in curves
    ax.plot(coordinates(c)...,color=:black,linewidth=2, label="Levelset for c of u₀")
end
for c in curves_epsilon
    ax.plot(coordinates(c)...,color=:red,linestyle="--",linewidth=2, label="Levelset for c of uϵ")
end
for c in curves_predicted
    ax.plot(coordinates(c)...,color=:lightgreen,linewidth=2, label="Levelset for c of u₀ + ϵ̇u₀")
end
ax.set_title("c=$(round(c_best,digits=4))")
ax.legend();

## SEBA

In [ ]:
softThreshold(u,μ) = sign.(u) .* max.(abs.(u) .- μ, 0)
normalizeU(u) = u.*sign.(sum(u))/maximum(u*sign.(sum(u)))

We are now interested in the linear response of the partitions chosen by the SEBA algorithm. For this we note that 
1. If **U₀** is the matrix containing the initial Eigenfunctions, then, once converged, the algorithm returns a new set of functions **S** and an orthonormal matrix **O** such that, if **U₀** = **QR** is the QR decomposition, we have **S** = normalizeU(softThreshold(**U₀**(**R**^-1)**O**ᵀ,μ))
2. If **Udot** is the matrix containig the initial linear response of **U₀**, then for any matrix B we have that **UdotB** is the linear response of **U₀B** by linearity. 

In [ ]:
using LinearAlgebra

In [ ]:
# standard heuristic
μ = 0.99/grid_resolution;

In [ ]:
@time S, O = SEBA(u₀,μ=μ,returnR=true,sort=false);

In [ ]:
Sϵ, Oϵ = SEBA(uϵ,μ=μ,returnR=true,sort=false);

In [ ]:
#The SEBA outputs, but without thresholding and rescaling, so that they are comparable to S_dot
S′ = Matrix(qr(u₀).Q)*O'
Sϵ′ = Matrix(qr(uϵ).Q)*Oϵ';

In [ ]:
# This is the matrix we effectively multiply u₀ with
inv(qr(u₀).R)*O'

In [ ]:
inv(qr(uϵ).R)*Oϵ'

In [ ]:
S_dot = u_dot*inv(qr(u₀).R)*O';

In the following, there is another interesting point to be aware of: 

We are predicting **Uϵ**(**R**^-1)**O**ᵀ, where **R** and **O** come from (the SEBA algorithm applied to) **Udot**. What we are comparing it to, however, is **Uϵ**(**Rϵ**^-1)**Oϵ**ᵀ, so there is another source of error.

In [ ]:
#for S′ and Sϵ′, we have a linear response, namely S_dot 
plots = []
for i in 1:nev
    push!(plots,plot_u(ctx, S′[:,i],  
            grid_resolution, grid_resolution, 
            title="S"*string(i), 
            colorbar=colorbar, color=color, clims=(-maximum(abs.(S′[:,i])),maximum(abs.(S′[:,i]))),
            xticks=xticks, yticks=yticks))
    push!(plots,plot_u(ctx, S_dot[:,i], 
            grid_resolution, grid_resolution, 
            title="̇S"*string(i), 
            colorbar=colorbar, color=color, clims=(-maximum(abs.(S_dot[:,i])),maximum(abs.(S_dot[:,i]))), 
            xticks=xticks, yticks=yticks))
    push!(plots,plot_u(ctx, S′[:,i] + ϵ*S_dot[:,i],  
            grid_resolution, grid_resolution, 
            title="S"*string(i)*" + $(ϵ)̇S"*string(i), 
            colorbar=colorbar, color=color, clims=(-maximum(abs.(S′[:,i] + ϵ*S_dot[:,i])),maximum(abs.(S′[:,i] + ϵ*S_dot[:,i]))), 
            xticks=xticks, yticks=yticks))
    push!(plots,plot_u(ctx, Sϵ′[:,i],  
            grid_resolution, grid_resolution, 
            title="Sϵ"*string(i), 
            colorbar=colorbar, color=color, clims=(-maximum(abs.(Sϵ′[:,i])),maximum(abs.(Sϵ′[:,i]))), 
            xticks=xticks, yticks=yticks))
end
Plots.plot(plots..., fmt=:png, dpi=500,layout=(nev,4),
            xtickfontsize=2,ytickfontsize=2,xguidefontsize=2,yguidefontsize=2,legendfontsize=5,titlefontsize=5)

In [ ]:
# to get a prediction for Sϵ from S and S_dot, we simply apply soft thresholding and normalization to the previous example
plots = []
for i in 1:nev
    push!(plots,plot_u(ctx, S[:,i],  
            grid_resolution, grid_resolution, 
            title="S"*string(i), 
            colorbar=colorbar, color=color, clims=(-1,1),
            xticks=xticks, yticks=yticks))
    push!(plots,plot_u(ctx, S_dot[:,i],  
            grid_resolution, grid_resolution, 
            title="̇S"*string(i), 
            colorbar=colorbar, color=color, clims=(-maximum(abs.(S_dot[:,i])),maximum(abs.(S_dot[:,i]))),
            xticks=xticks, yticks=yticks))
    push!(plots,plot_u(ctx, normalizeU(softThreshold(S′[:,i] + ϵ*S_dot[:,i],μ)),  
            grid_resolution, grid_resolution, 
            title="S"*string(i)*" + $(ϵ)̇S"*string(i), 
            colorbar=colorbar, color=color, clims=(-1,1),
            xticks=xticks, yticks=yticks))
    push!(plots,plot_u(ctx, Sϵ[:,i],  
            grid_resolution, grid_resolution, 
            title="Sϵ"*string(i), 
            colorbar=colorbar, color=color, clims=(-1,1),
            xticks=xticks, yticks=yticks))
end
Plots.plot(plots..., fmt=:png, dpi=500,layout=(nev,4),
            xtickfontsize=2,ytickfontsize=2,xguidefontsize=2,yguidefontsize=2,legendfontsize=5,titlefontsize=5)

In [ ]:
#again, select one index for closer inspection
ev = 2

In [ ]:
plots = []
push!(plots,plot_u(ctx, S′[:,ev],  
        grid_resolution, grid_resolution, 
        title="S"*string(ev), 
        colorbar=colorbar, color=color, clims=(-maximum(abs.(S′[:,ev])),maximum(abs.(S′[:,ev]))),
            xticks=xticks, yticks=yticks))
push!(plots,plot_u(ctx, S_dot[:,ev],  
        grid_resolution, grid_resolution, 
        title="̇S"*string(ev), 
        colorbar=colorbar, color=color, clims=(-maximum(abs.(S_dot[:,ev])),maximum(abs.(S_dot[:,ev]))),
            xticks=xticks, yticks=yticks))
push!(plots,plot_u(ctx, S′[:,ev] + ϵ*S_dot[:,ev],  
        grid_resolution, grid_resolution, 
        title="S"*string(ev)*" + $(ϵ)̇S"*string(ev), 
        colorbar=colorbar, color=color, clims=(-maximum(abs.(S′[:,ev] + ϵ*S_dot[:,ev])),maximum(abs.(S′[:,ev] + ϵ*S_dot[:,ev]))),
            xticks=xticks, yticks=yticks))
push!(plots,plot_u(ctx, Sϵ′[:,ev],  
        grid_resolution, grid_resolution, 
        title="Sϵ"*string(ev), 
        colorbar=colorbar, color=color, clims=(-maximum(abs.(Sϵ′[:,ev])),maximum(abs.(Sϵ′[:,ev]))),
            xticks=xticks, yticks=yticks))
Plots.plot(plots..., fmt=:png, dpi=500,layout=(2,2),
            xtickfontsize=2,ytickfontsize=2,xguidefontsize=2,yguidefontsize=2,legendfontsize=5,titlefontsize=5)

In [ ]:
plots = []
push!(plots,plot_u(ctx, S[:,ev],  
        grid_resolution, grid_resolution, 
        title="S"*string(ev), 
        colorbar=colorbar, color=color, clims=(-1,1),
            xticks=xticks, yticks=yticks))
push!(plots,plot_u(ctx, S_dot[:,ev],  
        grid_resolution, grid_resolution, 
        title="̇S"*string(ev), 
        colorbar=colorbar, color=color, clims=(-maximum(abs.(S_dot[:,ev])),maximum(abs.(S_dot[:,ev]))),
            xticks=xticks, yticks=yticks))
push!(plots,plot_u(ctx, normalizeU(softThreshold(S′[:,ev] + ϵ*S_dot[:,ev],μ)),  
        grid_resolution, grid_resolution, 
        title="S"*string(ev)*" + $(ϵ)̇S"*string(ev), 
        colorbar=colorbar, color=color, clims=(-1,1),
            xticks=xticks, yticks=yticks))
push!(plots,plot_u(ctx, Sϵ[:,ev],  
        grid_resolution, grid_resolution, 
        title="Sϵ"*string(ev), 
        colorbar=colorbar, color=color, clims=(-1,1),
            xticks=xticks, yticks=yticks))
Plots.plot(plots..., fmt=:png, dpi=500,layout=(2,2),
            xtickfontsize=2,ytickfontsize=2,xguidefontsize=2,yguidefontsize=2,legendfontsize=5,titlefontsize=5)

## Contourplots for SEBA

Levelsets are a better way to actually asses the quality of the predictions visually.

In [ ]:
best_levelset, best_value = get_minimal_levelset(
    ctx,S[:,ev],dynamic_cheeger_value,
    n_candidates=1000,min=0)
c_best = level(best_levelset)
print("Dynamic Cheeger value:\t",best_value,"\n")
print("Levelset:\t\t",c_best,"\n")

In [ ]:
plt.figure(figsize=(7,7), dpi=100)
ax = plt.axes()
ax.set(xlim=(0, 1), ylim=(0, 1), xticks=(0,1), yticks=(0,1))
plot_vectorfield(get_levelset_evolution(ctx,normalizeU(S′[:,ev]),S_dot[:,ev])...,ax,scale=0.2,step=4)
curves           = lines(get_levelset(ctx, S[:,ev],c_best))
curves_epsilon   = lines(get_levelset(ctx, Sϵ[:,ev],c_best))
curves_predicted = lines(get_levelset(ctx, normalizeU(softThreshold(S′[:,ev] + ϵ*S_dot[:,ev],μ)),c_best))

for c in curves
    ax.plot(coordinates(c)...,color=:black,linewidth=2, label="Levelset for c of S")
end
for c in curves_epsilon
    ax.plot(coordinates(c)...,color=:red,linestyle="--",linewidth=2, label="Levelset for c of Sϵ")
end
for c in curves_predicted
    ax.plot(coordinates(c)...,color=:lightgreen,linewidth=2, label="Levelset for c of S + ϵ̇S")
end
ax.set_title("c=$(round(c_best,digits=4))")
ax.legend(loc=1);